In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy as np, textblob, string

In [3]:
# Create a comment object
class Comment:
    def __init__(self, label, text):
        self.label = label
        self.text = text
        self.split_text = self.text.split()
  
    def __iter__(self):
        for i in range [0:len(self.split_text)-1] :
            return(self.split_text[i])
    
    def __next__(self):
        if i < len(self.split_text)-1:
            return self.split_text[i]
        else:
            raise StopIteration  # Done iterating.
        

In [4]:
# Open the file and create comment objects with the label and comment

import csv

raw_comments =[]

with open("Project work/Shortened training set.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        raw_comments.append(Comment(row['label'],row['comment']))

In [5]:
#Check it worked
print(raw_comments[10].label,raw_comments[10].text)

0 I think a significant amount would be against spending their tax dollars on other people.


Remove all stopwords from the split_text, but keep them in the original text. <br>
Add all comments with more than 3 words (after removing stopwords) to a new list comments

In [6]:
comments =[]

from nltk.corpus import stopwords

for comment in raw_comments:
    for word in comment.split_text:
        if word in stopwords.words('english'):
            comment.split_text.remove(word)
    if len(comment.split_text) >3:
        comments.append(comment)
        
        

In [7]:
print(comments[100].text, comments[100].split_text)

I figure the alt-Right would have a field day. ['I', 'figure', 'alt-Right', 'would', 'a', 'field', 'day.']


In [8]:
print(len(comments))

3703


In [9]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

sentim_analy = SentimentAnalyzer()

sentim_analy.classify(comments[0].split_text)


C:\Users\Jamie\PycharmProjects\DevProject\venv\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


AttributeError: 'NoneType' object has no attribute 'classify'

Create TextBlobs of all of the textual comments. This is useful for extracting a lot of information on the text, such as sentiment, nouns, verbs,word and noun frequencies, inflection, spelling corrections, etc.

In [10]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer


In [12]:
comment_blobs = []
for comment in comments:
    #Leave out the analyzer to default to the pattern analyser
    comment_blobs.append(TextBlob(comment.text, analyzer=NaiveBayesAnalyzer() ))

In [13]:
print(comment_blobs[1000])

[](/yeah) I have one answer, and one answer only: YEEEEEAAAAH!


Part of speech lists, giving each word in the sentence and what it is (i.e. noun, verb, adverb, etc. <br>
Sentiment lists, giving the polarity (between -1 and 1) and subjectivity (0.0 being very objective 1.0 being very subjective)

In [14]:
comment_pos = []
comment_sentiment =[]
comment_words = []
comment_sentences = []
comment_noun_phrases = []

for blob in comment_blobs:
    comment_pos.append(blob.tags)
    comment_sentiment.append(blob.sentiment)
    comment_words.append(blob.words)
    comment_sentences.append(blob.sentences)
    comment_noun_phrases.append(blob.noun_phrases)

In [ ]:
ind = 11
print(comment_blobs[ind], comment_sentiment[ind], comment_words[ind], comment_sentences[ind], comment_noun_phrases[ind])

because it's what really bothers him... and it's a sign of weakness. Sentiment(polarity=0.2, subjectivity=0.2) ['because', 'it', "'s", 'what', 'really', 'bothers', 'him', 'and', 'it', "'s", 'a', 'sign', 'of', 'weakness'] [Sentence("because it's what really bothers him... and it's a sign of weakness.")] []


In [ ]:
alt_nouns = []
from textblob.np_extractors import ConllExtractor

extractor = ConllExtractor
thisblob = TextBlob(comments[ind].text, np_extractor=extractor)
print(thisblob.noun_phrases)